In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('contacts.json')
df_grouped = df[["Id","Email","Phone","OrderId"]].melt(id_vars="Id").groupby("value").agg(lambda x: list(x))

In [6]:
L = list(df_grouped["Id"][1:])

## Using Sets

In [ ]:
import itertools

LL = set(itertools.chain.from_iterable(L)) 

for each in LL:
  components = [x for x in L if each in x]
  for i in components:
    L.remove(i)
  L += [list(set(itertools.chain.from_iterable(components)))]

## Using Networks

In [8]:
import networkx as nx
from networkx.algorithms.components.connected import connected_components

def to_graph(l):
    G = nx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(zip(part[:-1], part[1:]))
    return G

G = to_graph(L)
connected_list = [list(i) for i in connected_components(G)]

## Merging Lists

In [51]:
temp_df = pd.DataFrame({'A':connected_list,
'B':connected_list,'GroupID': range(0,len(connected_list))})
temp_df = temp_df.explode('A').sort_values('A').reset_index().drop("index", axis=1).rename(columns={"A":"Id", "B":"trace"})

In [53]:
full_df = temp_df.join(df["Contacts"], how="left", on="Id")
full_df["contact_count"] = full_df.groupby('GroupID')['Contacts'].transform(sum)